In [ ]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import gradio as gr
import pandas as pd
from finvizfinance.quote import finvizfinance

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import gradio as gr
import pandas as pd
from datetime import datetime
 

# Download VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()



# Define a function to calculate sentiment
def extract_score(text):
    score = sid.polarity_scores(text)
    compound = score['compound']
    sentiment = 'neutral'
    if compound >= 0.05:
        sentiment = "positive"
    elif compound <= -0.05:
        sentiment = "negative"
    return sentiment

# Function to process the stock ticker and generate synopsis
def process_ticker(ticker):
    ticker = ticker.upper()
    stock = finvizfinance(ticker)
    news_df = stock.ticker_news()

 # Add predicted sentiment column
    news_df["Predicted Sentiment"] = news_df["Title"].apply(extract_score)
    
    # Generate a synopsis summary of sentiment
    sentiment_counts = news_df["Predicted Sentiment"].value_counts()
    synopsis = sentiment_counts.to_string()  
    
    return news_df[["Title", "Predicted Sentiment"]], synopsis  

In [ ]:
# Function to clear all outputs
def clear_data():
    return "", pd.DataFrame(columns=["Title", "Predicted Sentiment"])

# Gradio Interface
with gr.Blocks(theme=gr.themes.Glass(primary_hue="blue")) as app:
    gr.Markdown("# 📈 Headline News Sentiment Analyzer")
    gr.Markdown(
        """
        Analyze the sentiment of recent news headlines for a given stock ticker.
        Just enter a valid ticker symbol (e.g., `AAPL`, `GOOGL`, `TSLA`) to see the predicted sentiment of recent news articles.
        """
    )
    with gr.Row():
        ticker_input = gr.Textbox(
            label="**Enter Stock Ticker**",  # Bold label for clarity
            placeholder="e.g., AAPL",
            lines=1,
        )
    
    with gr.Row():
        analyze_button = gr.Button("Analyze Sentiment")
        clear_button = gr.Button("Clear")

    sentiment_table = gr.DataFrame(
        headers=["Title", "Predicted Sentiment"],
        label="News Sentiment Analysis Results",
        datatype=["str", "str"],
    )
    synopsis_box = gr.Textbox(
        label="Sentiment Synopsis",
        placeholder="Summary of sentiment counts will appear here.",
        lines=3,
        interactive=False,  # Read-only
    )

    # Button actions
    analyze_button.click(
        process_ticker, 
        inputs=[ticker_input], 
        outputs=[sentiment_table, synopsis_box]
    )
    clear_button.click(
        clear_data, 
        inputs=[], 
        outputs=[ticker_input, sentiment_table, synopsis_box]
    )

# Custom CSS to fix label readability
app.css = """
/* Adjust font size and weight for the textbox label */
.gr-textbox .gr-label {
    font-size: 18px !important;
    font-weight: bold !important;
    color: #000 !important;
}
/* Style for the synopsis box */
.gr-textbox {
    width: 50%;  /* Adjust width to make it smaller */
    margin: 0 auto;  /* Center align */
}
"""

# Launch the app
app.launch()